# Superstore Sales Analytics (Python Cleaning Phase)
## Step 1: Load Dataset
In this step, we load the raw Superstore dataset into a pandas DataFrame and perform basic checks to confirm the data is loaded correctly.


In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = "Superstore_Sales_Data.csv"

# Load CSV into a pandas DataFrame
# encoding="ISO-8859-1" is used to handle special characters in the file
df = pd.read_csv(file_path, encoding="ISO-8859-1")

# Preview first 5 rows
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [3]:
# Check dataset shape (rows, columns)
print("Rows, Columns:", df.shape)

# Display column names
print("\nColumn Names:\n", df.columns)

Rows, Columns: (9994, 21)

Column Names:
 Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')


## Step 2A: Check Date Columns (Before Conversion)
We will first check whether Order Date and Ship Date are currently stored as text.


In [5]:
# Check current data types of date columns
df[["Order Date", "Ship Date"]].dtypes

Order Date    object
Ship Date     object
dtype: object

## Step 2B: Convert Order Date and Ship Date to Datetime
Currently, the `Order Date` and `Ship Date` columns are stored as text (`object` type).
In this step, we convert them into proper datetime format so that we can perform time-based analysis.


In [6]:
# Convert Order Date from text (object) to datetime
df["Order Date"] = pd.to_datetime(df["Order Date"], format="%m/%d/%Y")

# Convert Ship Date from text (object) to datetime
df["Ship Date"] = pd.to_datetime(df["Ship Date"], format="%m/%d/%Y")

# Confirm the conversion worked
df[["Order Date", "Ship Date"]].dtypes

Order Date    datetime64[ns]
Ship Date     datetime64[ns]
dtype: object

## Step 2C: Create Shipping Duration Feature (ship_days)
In this step, we calculate how many days it took to ship each order.
This will help us analyze operational performance like shipping speed by ship mode.


In [8]:
# Create a new column: ship_days
# This calculates the number of days between Ship Date and Order Date
df["ship_days"] = (df["Ship Date"] - df["Order Date"]).dt.days

# Preview the new column
df[["Order Date", "Ship Date", "ship_days"]].head()

,Order Date,Ship Date,ship_days
0,2016-11-08,2016-11-11,3
1,2016-11-08,2016-11-11,3
2,2016-06-12,2016-06-16,4
3,2015-10-11,2015-10-18,7
4,2015-10-11,2015-10-18,7


In [9]:
# Check minimum and maximum shipping days
df["ship_days"].min(), df["ship_days"].max()

(0, 7)

## Step 3: Create Time Features for Trend Analysis
To analyze sales and profit trends over time, we need time-related columns.
In this step, we extract Year, Month, Quarter, and Year-Month from the Order Date.


In [11]:
# Extract Year from Order Date
df["order_year"] = df["Order Date"].dt.year

# Extract Month number from Order Date (1 = Jan, 12 = Dec)
df["order_month"] = df["Order Date"].dt.month

# Extract Quarter from Order Date (Q1, Q2, Q3, Q4)
df["order_quarter"] = df["Order Date"].dt.quarter

# Create Year-Month format for monthly trend analysis (example: 2016-11)
df["order_year_month"] = df["Order Date"].dt.to_period("M").astype(str)

# Preview the new columns
df[["Order Date", "order_year", "order_month", "order_quarter", "order_year_month"]].head()

,Order Date,order_year,order_month,order_quarter,order_year_month
0,2016-11-08,2016,11,4,2016-11
1,2016-11-08,2016,11,4,2016-11
2,2016-06-12,2016,6,2,2016-06
3,2015-10-11,2015,10,4,2015-10
4,2015-10-11,2015,10,4,2015-10


In [12]:
df["order_year_month"].nunique()

48

## Step 4: Create Business Features (Profit Margin % and Discount Flag)
In this step, we create additional business metrics:
1) Profit Margin % to measure profitability per order line.
2) Discount Flag to identify whether a discount was applied.
These features will be helpful for deeper business analysis and dashboard insights.

In [14]:
# Create Profit Margin %
# Using np.where to avoid division by zero (if Sales is 0)
df["profit_margin_pct"] = np.where(df["Sales"] != 0, (df["Profit"] / df["Sales"]) * 100, 0)

# Round profit margin to 2 decimal places (clean for Excel / Power BI / SQL)
df["profit_margin_pct"] = df["profit_margin_pct"].round(2)

# Create Discount Flag:
# 0 = No discount, 1 = Discount applied
df["discount_flag"] = np.where(df["Discount"] > 0, 1, 0)

# Preview the new columns
df[["Sales", "Profit", "profit_margin_pct", "Discount", "discount_flag"]].head()

,Sales,Profit,profit_margin_pct,Discount,discount_flag
0,261.9600,41.9136,16.00,0.00,0
1,731.9400,219.5820,30.00,0.00,0
2,14.6200,6.8714,47.00,0.00,0
3,957.5775,-383.0310,-40.00,0.45,1
4,22.3680,2.5164,11.25,0.20,1


In [15]:
# Check profit margin summary
df["profit_margin_pct"].describe()

count    9994.000000
mean       12.031390
std        46.675436
min      -275.000000
25%         7.500000
50%        27.000000
75%        36.250000
max        50.000000
Name: profit_margin_pct, dtype: float64

In [16]:
df["discount_flag"].value_counts()

discount_flag
1    5196
0    4798
Name: count, dtype: int64

## Step 5: Final Checks and Export Clean Dataset
In this step, we perform final checks and export the cleaned dataset.
This exported file will be used in SQL and Power BI for analysis and dashboarding.

In [17]:
# Check for missing values in each column
df.isnull().sum()

Row ID               0
Order ID             0
Order Date           0
Ship Date            0
Ship Mode            0
Customer ID          0
Customer Name        0
Segment              0
Country              0
City                 0
State                0
Postal Code          0
Region               0
Product ID           0
Category             0
Sub-Category         0
Product Name         0
Sales                0
Quantity             0
Discount             0
Profit               0
ship_days            0
order_year           0
order_month          0
order_quarter        0
order_year_month     0
profit_margin_pct    0
discount_flag        0
dtype: int64

In [18]:
# Export cleaned dataset to a new CSV file
output_file = "superstore_clean.csv"
df.to_csv(output_file, index=False)